In [111]:
# import all prerequisites
from PyFoam.Applications.CloneCase import CloneCase
from PyFoam.Applications.ClearCase import ClearCase
from PyFoam.Applications.CreateBoundaryPatches import CreateBoundaryPatches
from PyFoam.Applications.ChangeBoundaryName import ChangeBoundaryName
from PyFoam.Applications.ChangeBoundaryType import ChangeBoundaryType
from PyFoam.Applications.WriteDictionary import WriteDictionary
from PyFoam.Applications.Runner import Runner
from PyFoam.Applications.MeshUtilityRunner import MeshUtilityRunner
from PyFoam.Applications.ClearInternalField import ClearInternalField
from PyFoam.Applications.Decomposer import Decomposer
import shutil, os, sys
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [36]:
sandia_path = "/home/pavel.kholodov/WORK/OpenFOAM/Benchmark/combustion/SandiaFlameD/flame/CaseB/B2_1X"
newcase = "Mixing"
meshfile = "Chamber_2Daxi.msh"

In [37]:
%%capture capt
# Close case from existing one
CloneCase(args=["--no-vcs",sandia_path,newcase,"--no-pyfoam","--add-item=0.orig","--force"])

In [38]:
%%capture capt
# Clear case
#ClearCase(args=["-h"])
ClearCase(args=["--remove-analyzed","--no-pyfoam","--additional=Mixing.foam","--additional=0",newcase])

In [103]:
%%capture capt
Runner(args=["--silent","fluentMeshToFoam","-case",os.path.join(os.getcwd(),newcase),meshfile])

In [104]:
%%capture capt
#shutil.copytree("./Mixing/0.orig","./Mixing/0")
ClearCase(args=["--remove-analyzed","--no-pyfoam","--additional=Mixing.foam",newcase])

In [105]:
%%capture capt
#shutil.copytree(os.path.join(newcase,"0.orig"),os.path.join(newcase,"0"))
# define BC names
mesh_bc = {'fuel_inlet':'FUEL_INLET',\
           'oxidizer_inlet':'OX_INLET',\
           'outlet':'OUTLET',\
           'opening':'ATMOSPHERE',\
           'coflow':'COFLOW',
           'walls':['BOTTOM','CHAMBER_WALL','NOZZLE_WALL','JET_WALL'],
           'sides':['FRONT','BACK']}
case_bc = {'fuel_inlet':'fuel_inlet',\
           'oxidizer_inlet':'ox_inlet',\
           'outlet':'outlet',\
           'opening':'atmosphere',\
           'coflow':'coflow',\
           'walls':['bottom','chamber_wall','nozzle_wall','jet_wall'],\
           'sides':['front','back']}
#Change names
ChangeBoundaryName(args=[newcase,mesh_bc['oxidizer_inlet'],case_bc['oxidizer_inlet']])
ChangeBoundaryName(args=[newcase,mesh_bc['fuel_inlet'],case_bc['fuel_inlet']])
for i in range(len(mesh_bc['walls'])):
    ChangeBoundaryName(args=[newcase,mesh_bc['walls'][i],case_bc['walls'][i]])

ChangeBoundaryName(args=[newcase,mesh_bc['coflow'],case_bc['coflow']])
ChangeBoundaryName(args=[newcase,mesh_bc['opening'],case_bc['opening']])
ChangeBoundaryName(args=[newcase,mesh_bc['outlet'],case_bc['outlet']])
for i in range(len(mesh_bc['sides'])):
    ChangeBoundaryName(args=[newcase,mesh_bc['sides'][i],case_bc['sides'][i]])

In [106]:
%%capture capt
# Change boundary type to appropriate
bc_types = {'fuel_inlet':'patch',\
            'oxidizer_inlet':'patch',\
            'outlet':'patch',\
            'opening':'patch',\
            'coflow':'patch',\
            'sides':'wedge'}
for bcname in [case_bc['oxidizer_inlet'],case_bc['fuel_inlet'],case_bc['outlet'],case_bc['opening']]:
    ChangeBoundaryType(args=[newcase,bcname,'patch'])
ChangeBoundaryType(args=[newcase,"front","wedge"])
for bc in case_bc['sides']:
    ChangeBoundaryType(args=[newcase,bc,bc_types['sides']])

In [43]:
%%capture capt
# Delete unneeded BC files
for i in ['CH4','OH','G','H2O','CO2','CO','O','H']:
    os.remove("./"+newcase+"/0/"+i)

In [44]:
%%capture capt
# Clear all boundary conditions from old names
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/U")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/p")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/T")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/k")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/epsilon")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/nut")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/alphat")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/O2")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/H2")])
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/N2")])

In [88]:
CreateBoundaryPatches(args=["--clear-unused","--overwrite",os.path.join(os.getcwd(),newcase,"0/Ydefault")])

In [45]:
%%capture capt
# BC for velocity
# Need to add somehow the table entry for inlets
#Wall BC
CreateBoundaryPatches(args=["--overwrite","--default={'type':'fixedValue','value':'uniform (0 0 0)'}",os.path.join(os.getcwd(),newcase,"0/U")])
# Inlet BC
for bcname in [case_bc['oxidizer_inlet'],case_bc['fuel_inlet']]:
    CreateBoundaryPatches(args=["--overwrite","--filter="+bcname,"--default={'type':'flowRateInletVelocity'}",os.path.join(os.getcwd(),newcase,"0/U")])
# Outlet BC
CreateBoundaryPatches(args=["--overwrite","--filter=coflow","--default={'type':'zeroGradient'}",os.path.join(os.getcwd(),newcase,"0/U")])
CreateBoundaryPatches(args=["--overwrite","--filter=outlet","--default={'type':'pressureInletOutletVelocity','value':'uniform (0 0 0)'}",os.path.join(os.getcwd(),newcase,"0/U")])
CreateBoundaryPatches(args=["--overwrite","--filter=outlet","--default={'type':'pressureInletOutletVelocity','value':'uniform (0 0 0)'}",os.path.join(os.getcwd(),newcase,"0/U")])
# Wedges
CreateBoundaryPatches(args=["--overwrite","--filter=front","--default={'type':'wedge'}",os.path.join(os.getcwd(),newcase,"0/U")])
CreateBoundaryPatches(args=["--overwrite","--filter=back","--default={'type':'wedge'}",os.path.join(os.getcwd(),newcase,"0/U")])

In [46]:
%%capture capt
# BC for pressure
# Wall BC
CreateBoundaryPatches(args=["--overwrite","--default={'type':'zeroGradient'}",os.path.join(os.getcwd(),newcase,"0/p")])
# Inlet BC
CreateBoundaryPatches(args=["--overwrite","--filter=ox_inlet","--default={'type':'zeroGradient'}",os.path.join(os.getcwd(),newcase,"0/p")])
#CreateBoundaryPatches(args=["--overwrite","--filter=fuel_inlet","--default={'type':'zeroGradient'}",os.path.join(os.getcwd(),newcase,"0/U")])
# Outlet BC
CreateBoundaryPatches(args=["--overwrite","--filter=outlet","--default={'type':'pressureInletOutletVelocity','value':'uniform 98066.5'}",os.path.join(os.getcwd(),newcase,"0/p")])
CreateBoundaryPatches(args=["--overwrite","--filter=coflow","--default={'type':'pressureInletOutletVelocity','value':'uniform 98066.5'}",os.path.join(os.getcwd(),newcase,"0/p")])

In [59]:
ClearInternalField(args=['--value=0.0',os.path.join(os.getcwd(),newcase,"0/H2")])
ClearInternalField(args=['--value=0.23',os.path.join(os.getcwd(),newcase,"0/O2")])
ClearInternalField(args=['--value=0.77',os.path.join(os.getcwd(),newcase,"0/N2")])
ClearInternalField(args=['--value=0.0',os.path.join(os.getcwd(),newcase,"0/Ydefault")])

ClearInternalField(args=['--value=293.0',os.path.join(os.getcwd(),newcase,"0/T")])
ClearInternalField(args=['--value=(0 0 0)',os.path.join(os.getcwd(),newcase,"0/U")])
ClearInternalField(args=['--value=98066.5',os.path.join(os.getcwd(),newcase,"0/p")])

ClearInternalField(args=['--value=5.0',os.path.join(os.getcwd(),newcase,"0/k")])
ClearInternalField(args=['--value=32.0',os.path.join(os.getcwd(),newcase,"0/epsilon")])

In [47]:
%%capture capt
# BC for H2,O2,N2 components
files = ['O2','H2','N2']
# Wall BC
for i in files:
    CreateBoundaryPatches(args=["--overwrite","--default={'type':'zeroGradient'}",\
                                os.path.join(os.getcwd(),newcase,"0",i)])
# Inlet BC
# oxidizer inlet
values = ['1.0','0.0','0.0']
for i in range(len(files)):
    CreateBoundaryPatches(args=["--overwrite","--filter="+case_bc['oxidizer_inlet'],\
                                "--default={'type':'fixedValue','value':'uniform %s'}"%(values[i]),\
                                os.path.join(os.getcwd(),newcase,"0",files[i])])
# fuel inlet
values = ['0.0','1.0','0.0']
CreateBoundaryPatches(args=["--overwrite","--filter="+case_bc['fuel_inlet'],\
                            "--default={'type':'fixedValue','value':'uniform %s'}"%(values[i]),\
                            os.path.join(os.getcwd(),newcase,"0",files[i])])
# Outlet BC - Atmosphere
files = ['O2','H2','N2']
values = ['0.23','0.0','0.77']
for i in range(len(files)):
    CreateBoundaryPatches(args=["--overwrite","--filter="+case_bc['opening'],\
                                "--default={'type':'fixedValue','value':'uniform %s'}"%(values[i]),\
                                os.path.join(os.getcwd(),newcase,"0",files[i])])
    CreateBoundaryPatches(args=["--overwrite","--filter="+case_bc['outlet'],\
                                "--default={'type':'fixedValue','value':'uniform %s'}"%(values[i]),\
                                os.path.join(os.getcwd(),newcase,"0",files[i])])

In [89]:
# BC for Ydefault components
CreateBoundaryPatches(args=["--overwrite","--default={'type':'zeroGradient'}",os.path.join(os.getcwd(),newcase,"0/Ydefault")])
for bcname in [case_bc['oxidizer_inlet'],case_bc['fuel_inlet'],case_bc['opening'],case_bc['outlet']]:
    CreateBoundaryPatches(args=["--overwrite","--filter="+bcname,\
                                "--default={'type':'fixedValue','value':'uniform 0.0'}",\
                                os.path.join(os.getcwd(),newcase,"0/Ydefault")])


In [49]:
%%capture capt
# BC for temperature
CreateBoundaryPatches(args=["--overwrite","--default={'type':'fixedValue','value':'uniform 293'}",\
                            os.path.join(os.getcwd(),newcase,"0/T")])

for bcname in [case_bc['opening'],case_bc['outlet']]:
    CreateBoundaryPatches(args=["--overwrite","--filter="+bcname,"--default={'type':'inletOutlet','value':'uniform 293'}",\
                                os.path.join(os.getcwd(),newcase,"0/T")])
"""
WriteDictionary(args=[os.path.join(os.getcwd(),newcase,"0/T"),\
                      "boundaryField['outlet']['value']","293"])
WriteDictionary(args=[os.path.join(os.getcwd(),newcase,"0/T"),\
                      "boundaryField['atmosphere']['value']","293"])
"""

'\nWriteDictionary(args=[os.path.join(os.getcwd(),newcase,"0/T"),                      "boundaryField[\'outlet\'][\'value\']","293"])\nWriteDictionary(args=[os.path.join(os.getcwd(),newcase,"0/T"),                      "boundaryField[\'atmosphere\'][\'value\']","293"])\n'

In [50]:
%%capture capt
# BC for alphat
CreateBoundaryPatches(args=["--overwrite","--default={'type':'compressible::alphatWallFunction','value':'$internalField'}",\
                            os.path.join(os.getcwd(),newcase,"0/alphat")])
# BC for inlets/outlets
for bcname in [case_bc['oxidizer_inlet'],case_bc['fuel_inlet'],case_bc['opening'],case_bc['outlet']]:
    CreateBoundaryPatches(args=["--overwrite","--filter="+bcname,\
                                "--default={'type':'calculated','value':'$internalField'}",\
                                os.path.join(os.getcwd(),newcase,"0/alphat")])

In [51]:
%%capture capt
# BC for k, epsilon, nut
CreateBoundaryPatches(args=["--overwrite","--default={'type':'kqRWallFunction','value':'uniform 0'}",\
                            os.path.join(os.getcwd(),newcase,"0/k")])
k_values = ['0.05','0.05','0.01','0.01']
for bcname in [case_bc['oxidizer_inlet'],case_bc['fuel_inlet']]:
    CreateBoundaryPatches(args=["--overwrite","--filter="+bcname,\
                                "--default={'type':'turbulentIntensityKineticEnergyInlet','value':'1'}",\
                                os.path.join(os.getcwd(),newcase,"0/k")])
for bcname in [case_bc['opening'],case_bc['outlet']]:
    CreateBoundaryPatches(args=["--overwrite","--filter="+bcname,\
                            "--default={'type':'inletOutlet','value':'uniform 0.001'}",\
                            os.path.join(os.getcwd(),newcase,"0/k")])
# BC for epsilon
mixing_length = [1.5e-4,7.6e-5,0.031,0.00923]
for bcname in [case_bc['oxidizer_inlet'],case_bc['fuel_inlet']]:
    CreateBoundaryPatches(args=["--overwrite","--filter="+bcname,\
                                "--default={'type':'turbulentMixingLengthDissipationRateInlet','value':'200'}",\
                                os.path.join(os.getcwd(),newcase,"0/epsilon")])

for bcname in [case_bc['opening'],case_bc['outlet']]:
    CreateBoundaryPatches(args=["--overwrite","--filter="+bcname,\
                            "--default={'type':'inletOutlet','value':'uniform 0.0003'}",\
                            os.path.join(os.getcwd(),newcase,"0/epsilon")])
# BC for nut
for bcname in case_bc['walls']:
    CreateBoundaryPatches(args=["--overwrite","--filter="+bcname,\
                        "--default={'type':'nutkWallFunction','value':'uniform 0'}",\
                        os.path.join(os.getcwd(),newcase,"0/nut")])
for bcname in [case_bc['oxidizer_inlet'],case_bc['fuel_inlet'],case_bc['opening'],case_bc['outlet']]:
    CreateBoundaryPatches(args=["--overwrite","--filter="+bcname,\
                        "--default={'type':'calculated','value':'uniform 0'}",\
                        os.path.join(os.getcwd(),newcase,"0/nut")])

In [102]:
shutil.rmtree(os.path.join(os.getcwd(),newcase,"0.mixing.orig"))
shutil.copytree(os.path.join(os.getcwd(),newcase,"0"),os.path.join(os.getcwd(),newcase,"0.mixing.orig"))

In [110]:
os.system("foamDictionary -entry 'startFrom' -set 'startTime' %s/system/controlDict 2>err"%(newcase))
os.system("foamDictionary -entry 'stopAt' -set 'endTime' %s/system/controlDict 2>err"%(newcase))
os.system("foamDictionary -entry 'endTime' -set '0.07' %s/system/controlDict 2>err"%(newcase))
os.system("foamDictionary -entry 'deltaT' -set '1e-7' %s/system/controlDict 2>err"%(newcase))
os.system("foamDictionary -entry 'writeControl' -set 'adjustableRunTime' %s/system/controlDict 2>err"%(newcase))
os.system("foamDictionary -entry 'writeInterval' -set '0.002' %s/system/controlDict 2>err"%(newcase))
os.system("foamDictionary -entry 'adjustTimeStep' -set 'yes' %s/system/controlDict 2>err"%(newcase))
os.system("foamDictionary -entry 'maxDeltaT' -set '0.0001' %s/system/controlDict 2>err"%(newcase))
os.system("foamDictionary -entry 'maxCo' -set '3' %s/system/controlDict 2>err"%(newcase))

os.system("foamDictionary -entry 'functions.minMaxT.writeControl' -set 'adjustableRunTime' %s/system/controlDict 2>err"%(newcase))
os.system("foamDictionary -entry 'functions.minMaxT.writeInterval' -set '1e-5' %s/system/controlDict 2>err"%(newcase))

os.system("foamDictionary -entry 'functions.minMaxP.writeControl' -set 'adjustableRunTime' %s/system/controlDict 2>err"%(newcase))
os.system("foamDictionary -entry 'functions.minMaxP.writeInterval' -set '1e-5' %s/system/controlDict 2>err"%(newcase))

os.system("foamDictionary -entry 'functions.volAveTemp.writeControl' -set 'adjustableRunTime' %s/system/controlDict 2>err"%(newcase))
os.system("foamDictionary -entry 'functions.volAveTemp.writeInterval' -set '1e-5' %s/system/controlDict 2>err"%(newcase))
os.system("foamDictionary -entry 'functions.volAveTemp.name' -set 'chamber' %s/system/controlDict 2>err"%(newcase))

0

In [92]:
os.system("foamDictionary -entry 'chemistry' -set 'off' %s/constant/chemistryProperties 2>err"%(newcase))

0

In [93]:
os.system("foamDictionary -entry 'ddtSchemes.default' -set 'Euler' %s/system/fvSchemes 2>err"%(newcase))
os.system("foamDictionary -entry 'PIMPLE.nOuterCorrectors' -set '30' %s/system/fvSolution 2>err"%(newcase))

0

#Could not execute for "actions" in topoSetDict
os.system("foamDictionary -entry 'actions' -set 'chamber' %s/system/topoSetDict 2>err"%(newcase))
#os.system("foamDictionary -entry 'actions.sourceInfo.p2' -set '(0.2 0.0 0.0)' %s/system/topoSetDict 2>err"%(newcase))
#os.system("foamDictionary -entry 'actions.sourceInfo.radius' -set '0.05' %s/system/topoSetDict 2>err"%(newcase))

In [109]:
Runner(args=["--silent","topoSet","-case",os.path.join(os.getcwd(),newcase)])

 PyFoam WARNING on line 42 of file /home/pavel.kholodov/anaconda2/lib/python2.7/site-packages/PyFoam/Infrastructure/ServerBase.py : No certficate file /home/pavel.kholodov/.pyFoam/foamServerCertificate.cert exists. Therefor no SSL-connection for the FoamServer possible
 To generate a private key:
    openssl genrsa -out /home/pavel.kholodov/.pyFoam/foamServerCertificate.key 2048 
Then generate the cerificate that is valid for 3 years with 
    openssl req -new -x509 -key /home/pavel.kholodov/.pyFoam/foamServerCertificate.key -out /home/pavel.kholodov/.pyFoam/foamServerCertificate.cert -days 1095 


In [114]:
Decomposer(args=["--method=simple","--n=(4,1,1)","--delta=1e-4","--no-decompose","./%s"%(newcase),"4"])